Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

InvoiceNo — номер транзакции
StockCode — код товара
Description — описание товара
Quantity — количество единиц товара, добавленных в заказ
InvoiceDate — дата транзакции 
UnitPrice — цена за единицу товара
CustomerID — id клиента
Country — страна, где проживает клиент
Данные можно скачать отсюда.

Прочитайте данные с кодировкой ISO-8859-1

In [1]:
import pandas as pd

In [2]:
#считываем данные
retail=pd.read_csv('D://KARPOV/5_data.csv.zip', compression = 'zip', encoding = 'ISO-8859-1')

In [3]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail_columns=retail.columns

In [5]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [6]:
retail.shape

(541909, 8)

In [7]:
#убираем дубликаты
without_dublicates = retail.drop_duplicates()

In [8]:
without_dublicates.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [9]:
#Данные содержат в себе записи как об успешных транзакциях, так и об отмененных.
#Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
#Сколько всего транзакций отменили пользователи? Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.

without_dublicates_cancelled = without_dublicates[without_dublicates['InvoiceNo'].str.contains('C')]

#9251 - столько транакций было отменено

In [10]:
without_dublicates_cancelled.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom


In [11]:
# Отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0
without_dublicates.query('Quantity > 0').count()

InvoiceNo      526054
StockCode      526054
Description    525462
Quantity       526054
InvoiceDate    526054
UnitPrice      526054
CustomerID     392732
Country        526054
dtype: int64

In [12]:
#Посчитайте число заказов для каждого пользователя (CustomerID) из Германии. 
#Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.
#Запишите полученные id пользователей в germany_top (только id).
germany_id = without_dublicates.query('Country == "Germany"')\
                  .groupby('CustomerID')\
                  .agg({'InvoiceNo':'nunique'})

In [13]:
q = germany_id.InvoiceNo.quantile(0.8)
germany_top=germany_id.query('InvoiceNo > @q').index

In [14]:
germany_top

Float64Index([12471.0, 12472.0, 12474.0, 12476.0, 12481.0, 12500.0, 12569.0,
              12600.0, 12619.0, 12621.0, 12626.0, 12647.0, 12662.0, 12705.0,
              12708.0, 12709.0, 12712.0, 12720.0],
             dtype='float64', name='CustomerID')

In [16]:
#Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения 
#и оставьте в данных записи только по интересующим нас юзерам. 
#Результирующий датафрейм запишите в top_retail_germany.
#1 способ
top_retail_germany= retail.query('CustomerID in @germany_top')

In [17]:
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


In [18]:
#2 способ
top_retail_germany_2=retail.loc[retail.CustomerID.isin(germany_top)]

In [27]:
top_retail_germany_2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


In [35]:
#Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
top_retail_germany_2.groupby('StockCode').agg({'Quantity':'count'}).sort_values('Quantity', ascending=False)
#Как мы видим, ТОП -2 (после POST) - 22326

,Quantity
StockCode,
POST,219
22326,63
22423,55
22328,45
22467,37
...,...
23008,1
22999,1
20717,1


In [ ]:
#Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [37]:
revenue=without_dublicates.Quantity * without_dublicates.UnitPrice

In [38]:
revenue

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Length: 536641, dtype: float64

In [41]:
without_dublicates['revenue']=revenue

<ipython-input-41-da67dc8561df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  without_dublicates['revenue']=revenue


In [ ]:
#Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. 
# качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа

In [51]:
without_dublicates=without_dublicates.groupby('InvoiceNo').agg({'Revenue':'sum'}).sort_values('Revenue', ascending=False).head(5)

In [52]:
#Топ -5 транзакций по сумме заказа
without_dublicates.index

Index(['581483', '541431', '574941', '576365', '556444'], dtype='object', name='InvoiceNo')